# Proyecto Final – Ciencia de Datos

### Predicción de Ventas Globales de Videojuegos

Este proyecto tiene como objetivo predecir las ventas globales (`Global_Sales`) de videojuegos utilizando técnicas de ciencia de datos. El análisis se basa en un dataset histórico con información sobre títulos, plataformas, géneros, puntuaciones de críticos y usuarios, entre otras variables. 

A lo largo del proyecto se aplicarán los siguientes pasos:
1. Limpieza y preprocesamiento de los datos
2. Selección de características (feature selection)
3. Entrenamiento de un modelo de regresión
4. Evaluación del modelo mediante métricas apropiadas
5. Conclusiones basadas en los resultados obtenidos


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [ ]:
url = "https://raw.githubusercontent.com/MarcosRomano13/proyecto-ciencia-datos-videojuegos/main/Dataset_final.csv"
df = pd.read_csv(url)

df.columns = df.columns.str.strip().str.replace(" ", "_")

df["Release_Date"] = pd.to_datetime(df["Release_Date"], errors="coerce")
df["Year_of_Release"] = df["Release_Date"].dt.year

df = df.dropna(subset=[
    "Global_Sales", "Critic_Score", "User_Score",
    "Genre", "Platform", "Year_of_Release"
])

df.loc[:, "User_Score"] = pd.to_numeric(df["User_Score"], errors="coerce")
df = df.dropna(subset=["User_Score"])

le_genre = LabelEncoder()
le_platform = LabelEncoder()
df["Genre"] = le_genre.fit_transform(df["Genre"])
df["Platform"] = le_platform.fit_transform(df["Platform"])


In [ ]:
X = df[["Critic_Score", "User_Score", "Genre", "Platform", "Year_of_Release"]]
y = df["Global_Sales"]

selector = SelectKBest(score_func=f_regression, k=5)
X_new = selector.fit_transform(X, y)

selected_features = X.columns[selector.get_support()]
print("Variables seleccionadas:", list(selected_features))


In [ ]:
X_selected = df[selected_features]
y = df["Global_Sales"]

X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)


In [ ]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error cuadrático medio (MSE): {mse:.2f}")
print(f"Coeficiente de determinación (R2): {r2:.2f}")


### Importancia de las variables en el modelo

In [ ]:
importances = pd.Series(model.feature_importances_, index=selected_features)
importances.sort_values().plot(kind='barh')
plt.title("Importancia de las variables en el modelo")
plt.xlabel("Importancia")
plt.ylabel("Variable")
plt.tight_layout()
plt.show()


### Comparación de ventas reales vs. predichas

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Ventas reales")
plt.ylabel("Ventas predichas")
plt.title("Ventas reales vs. predichas")
plt.tight_layout()
plt.show()


### Distribución de ventas globales

In [ ]:
sns.histplot(df["Global_Sales"], bins=50, kde=True)
plt.title("Distribución de ventas globales")
plt.xlabel("Millones de unidades")
plt.tight_layout()
plt.show()


### Conclusiones

Este proyecto permitió aplicar todo el flujo de trabajo típico en un problema real de ciencia de datos: desde la limpieza del dataset hasta el entrenamiento y validación de un modelo predictivo. Se utilizó un modelo de regresión basado en Random Forest para estimar las ventas globales de videojuegos a partir de variables numéricas y categóricas procesadas.

Las variables más influyentes seleccionadas por `SelectKBest` fueron:
- Puntuación de críticos (`Critic_Score`)
- Puntuación de usuarios (`User_Score`)
- Género (`Genre`)
- Plataforma (`Platform`)
- Año de lanzamiento (`Year_of_Release`)

Estos resultados no solo coinciden con la intuición (es razonable pensar que las buenas críticas y los géneros populares impactan en las ventas), sino que también nos permiten validar, con datos, que existen patrones aprovechables para estimar el rendimiento comercial de un juego.

El modelo logró capturar una parte significativa de la variabilidad, aunque no toda. Esto se refleja en un **R²** aceptable, pero no perfecto. Esto tiene sentido, ya que las ventas de un videojuego dependen de muchos factores no incluidos en el dataset, como:
- Publicidad y marketing
- Popularidad de la franquicia o personajes
- Fechas de lanzamiento (competencia, época del año)
- Exclusividad en plataformas
- Reputación del estudio desarrollador

A pesar de estas limitaciones, el modelo mostró un buen desempeño general, siendo fácil de implementar, rápido de entrenar y explicable. Esto lo convierte en un punto de partida útil para estudios de mercado en la industria del gaming.

### 🔮 Próximos pasos

- Aplicar validación cruzada para mejorar la robustez de los resultados.
- Comparar este modelo con otros como `XGBoost`, `GradientBoosting`, o redes neuronales.
- Incorporar más datos: campañas de marketing, valoraciones sociales, secuelas o precuelas, etc.
- Hacer tuning de hiperparámetros con GridSearch o RandomSearch para optimizar el rendimiento.

En resumen, este trabajo no solo cumple con los objetivos de implementación técnica, sino que también ofrece una mirada concreta sobre el potencial del análisis de datos en una industria tan dinámica como la de los videojuegos.
